
# Lead Measures Analysis

In [188]:


import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.formula.api as smf

# We brought the different performance related data in different tables and then merge them together as a pandas dataframe

## Data Preparation

In [172]:
df_final=pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(df_unit,df_battery,on=['work_day','Supervisor','wh_id','End Hour'],how='left'),df_direct,on=['work_day','Supervisor','wh_id','End Hour'],how='left'),df_sam,on=['work_day','Supervisor','wh_id','End Hour'],how='left'),df_pickput,on=['work_day','Supervisor','wh_id','End Hour'],how='left'),df_beginning,on=['work_day','Supervisor','wh_id','End Hour'],how='left'),df_ending,on=['work_day','Supervisor','wh_id','End Hour'],how='left')



In [173]:
df_final.isnull().sum()

wh_id                        0
work_day                     0
Supervisor                   0
End Hour                     0
Total units                  0
Avg. Battery Chg. Time    4427
Direct Time               2140
Indirect Time             2140
Lost Time                 2140
Tran over SAM                3
Total No. of Tran            3
Put units                   11
Picked units                11
Beg.Idle                  5669
End.Idle                  5430
dtype: int64

In [174]:
df_final=df_final.dropna(subset=['Direct Time','Put units'])
df_final['Avg. Battery Chg. Time'] = df_final['Avg. Battery Chg. Time'].replace(np.nan, 0)
df_final['Beg.Idle'] = df_final['Beg.Idle'].replace(np.nan, 0)
df_final['End.Idle'] = df_final['End.Idle'].replace(np.nan, 0)
df_final.isna().sum()

wh_id                     0
work_day                  0
Supervisor                0
End Hour                  0
Total units               0
Avg. Battery Chg. Time    0
Direct Time               0
Indirect Time             0
Lost Time                 0
Tran over SAM             0
Total No. of Tran         0
Put units                 0
Picked units              0
Beg.Idle                  0
End.Idle                  0
dtype: int64

In [175]:
df_final['pickput']=df_final['Put units']/df_final['Picked units']
df_final['SAM Overage']=df_final['Tran over SAM']/df_final['Total No. of Tran']
df_final.head()

,wh_id,work_day,Supervisor,End Hour,Total units,Avg. Battery Chg. Time,Direct Time,Indirect Time,Lost Time,Tran over SAM,Total No. of Tran,Put units,Picked units,Beg.Idle,End.Idle,pickput,SAM Overage
0,1,2020-07-20,425.0,15,91.0,0.0,57.409947,0.276666,2.313387,212.0,753.0,86.0,172.0,23.0,0.0,0.500000,0.281541
1,1,2020-07-15,25.0,4,350.0,0.0,48.815584,2.776041,7.202126,399.0,1878.0,352.0,275.0,20.0,0.0,1.280000,0.212460
2,1,2020-06-14,3301.0,0,74.0,0.0,52.862457,6.033333,1.075044,123.0,525.0,79.0,63.0,0.0,0.0,1.253968,0.234286
3,1,2020-07-16,620.0,14,400.0,0.0,51.698986,3.409803,2.037290,459.0,1709.0,392.0,224.0,0.0,0.0,1.750000,0.268578
6,1,2020-08-03,138.0,14,170.0,0.0,49.503309,1.649999,5.496694,537.0,1125.0,144.0,251.0,0.0,9.0,0.573705,0.477333


In [176]:
df_final['pickput'] = df_final['pickput'].replace(np.inf, 0)

In [177]:
df_fin=df_final[['Total units','Direct Time','Indirect Time','Lost Time','Picked units','Beg.Idle','End.Idle','Avg. Battery Chg. Time']]
corr=df_fin.corr()
corr.style.background_gradient(cmap='coolwarm')

,Total units,Direct Time,Indirect Time,Lost Time,Picked units,Beg.Idle,End.Idle,Avg. Battery Chg. Time
Total units,1.000000,0.319642,0.128404,-0.326335,0.628540,-0.042217,-0.059728,0.212003
Direct Time,0.319642,1.000000,-0.077406,-0.821537,0.436254,-0.109894,-0.282041,0.105467
Indirect Time,0.128404,-0.077406,1.000000,-0.158759,0.120008,0.211310,-0.081199,0.094200
Lost Time,-0.326335,-0.821537,-0.158759,1.000000,-0.376805,0.135581,0.321441,-0.105376
Picked units,0.628540,0.436254,0.120008,-0.376805,1.000000,-0.027084,-0.108132,0.234131
Beg.Idle,-0.042217,-0.109894,0.211310,0.135581,-0.027084,1.000000,-0.101288,-0.043730
End.Idle,-0.059728,-0.282041,-0.081199,0.321441,-0.108132,-0.101288,1.000000,-0.006947
Avg. Battery Chg. Time,0.212003,0.105467,0.094200,-0.105376,0.234131,-0.043730,-0.006947,1.000000


In [178]:
df_final['Total units']=np.log(df_final['Total units'])
df_final['Picked units']=np.log(df_final['Picked units'])
df_final['Direct Time']=np.log(df_final['Direct Time'])
df_final['Avg. Battery Chg. Time']=np.log(df_final['Avg. Battery Chg. Time'])
df_final['Beg.Idle']=np.log(df_final['Beg.Idle'])
df_final['End.Idle']=np.log(df_final['End.Idle'])
df_final['Indirect Time']=np.log(df_final['Indirect Time'])
df_final['Lost Time']=np.log(df_final['Lost Time'])
df_final['Picked units'] = df_final['Picked units'].replace(-np.inf, 0)
df_final['Direct Time'] = df_final['Direct Time'].replace(-np.inf, 0)
df_final['Indirect Time'] = df_final['Indirect Time'].replace(-np.inf, 0)
df_final['Lost Time'] = df_final['Lost Time'].replace(-np.inf, 0)
df_final['Beg.Idle'] = df_final['Beg.Idle'].replace([np.nan, -np.inf,np.inf], 0)
df_final['End.Idle'] = df_final['End.Idle'].replace([np.nan, -np.inf,np.inf], 0)
df_final['Avg. Battery Chg. Time'] = df_final['Avg. Battery Chg. Time'].replace([np.nan, -np.inf,np.inf], 0)
x=df_final[['Avg. Battery Chg. Time','Direct Time','Indirect Time','Lost Time','SAM Overage','Picked units','Beg.Idle','End.Idle']]
y=df_final['Total units']

C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [179]:
df_final.head()

,wh_id,work_day,Supervisor,End Hour,Total units,Avg. Battery Chg. Time,Direct Time,Indirect Time,Lost Time,Tran over SAM,Total No. of Tran,Put units,Picked units,Beg.Idle,End.Idle,pickput,SAM Overage
0,1,2020-07-20,425.0,15,4.510860,0.0,4.050218,-1.284944,0.838713,212.0,753.0,86.0,5.147494,3.135494,0.000000,0.500000,0.281541
1,1,2020-07-15,25.0,4,5.857933,0.0,3.888050,1.021026,1.974376,399.0,1878.0,352.0,5.616771,2.995732,0.000000,1.280000,0.212460
2,1,2020-06-14,3301.0,0,4.304065,0.0,3.967693,1.797300,0.072362,123.0,525.0,79.0,4.143135,0.000000,0.000000,1.253968,0.234286
3,1,2020-07-16,620.0,14,5.991465,0.0,3.945438,1.226655,0.711620,459.0,1709.0,392.0,5.411646,0.000000,0.000000,1.750000,0.268578
6,1,2020-08-03,138.0,14,5.135798,0.0,3.902040,0.500775,1.704147,537.0,1125.0,144.0,5.525453,0.000000,2.197225,0.573705,0.477333


In [180]:
df_final.isna().sum()

wh_id                     0
work_day                  0
Supervisor                0
End Hour                  0
Total units               0
Avg. Battery Chg. Time    0
Direct Time               0
Indirect Time             0
Lost Time                 0
Tran over SAM             0
Total No. of Tran         0
Put units                 0
Picked units              0
Beg.Idle                  0
End.Idle                  0
pickput                   0
SAM Overage               0
dtype: int64

## Regression Analysis

In [181]:
import statsmodels.api as sm
x=sm.add_constant(x)
model=sm.OLS(y,x)
results=model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            Total units   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     1330.
Date:                Thu, 13 Aug 2020   Prob (F-statistic):               0.00
Time:                        11:49:13   Log-Likelihood:                -5721.7
No. Observations:                4565   AIC:                         1.146e+04
Df Residuals:                    4556   BIC:                         1.152e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      3

## Exhaustive Feature Selection to identify top 3 features with highest r2 value

In [182]:
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

lr = LinearRegression()

efs = EFS(lr, 
          min_features=1,
          max_features=3,
          scoring='r2',
          cv=10)

efs.fit(x, y)

print('Best R2 score: %.2f' % efs.best_score_ * (-1))
print('Best subset:', efs.best_idx_)

Features: 129/129


Best subset: (2, 5, 6)


In [183]:
efs.best_score_

0.6669160939054221

In [184]:
efs.best_feature_names_

('Direct Time', 'SAM Overage', 'Picked units')

In [199]:
b=pd.DataFrame.from_dict(efs.get_metric_dict()).T
b.head()

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
0,"(0,)","[-0.047241003574984175, -0.02730601745532013, ...",-0.0568374,"(const,)",0.0515222,0.0693702,0.0231234
1,"(1,)","[0.02689257913008003, 0.023920859707815323, 0....",0.00576514,"(Avg. Battery Chg. Time,)",0.0582959,0.0784905,0.0261635
2,"(2,)","[0.18677520247888912, 0.20467021412595543, 0.2...",0.14706,"(Direct Time,)",0.0759161,0.102215,0.0340715
3,"(3,)","[0.06909762203370318, 0.03184655049847085, 0.0...",8.2176e-05,"(Indirect Time,)",0.0751538,0.101188,0.0337294
4,"(4,)","[0.07844497439419451, 0.17200652763126822, 0.1...",0.0785696,"(Lost Time,)",0.0728452,0.0980799,0.0326933


In [187]:
b[b['avg_score']==b['avg_score'].max()]


,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
103,"(2, 5, 6)","[0.689370687748552, 0.7033991004752793, 0.6799...",0.666916,"(Direct Time, SAM Overage, Picked units)",0.0296495,0.0399205,0.0133068


### From the analysis of this sample data, we are concluding that Direct Time, SAM Overage and Pickput ratio are the top three important lead measures  out of the 8 measures. These three together explains 67% variation of the efficiency of Arcadia Shipping Department's employee. 